### Задача 4

*Эту задачу можно выполнить в Питоне. Можно в отдельном ноутбуке.*

Проведите эксперимент по определению реального уровня значимости критерия для проверки гипотезы о незначимости коэффициента в гауссовской линейной модели, если на самом деле в данных присутствует гетероскедастичность. 
Для этого смоделируйте некоторым образом двумерные данные $x$ и посчитайте по ним ожидаемый отклик 
	$y(x) = \theta_0 + \theta_1 x^{(1)} + \theta_2 x^{(2)}$, где коэффициенты выберите по своему усмотрению, причем $\theta_2 = 0$. 
Зашумите набор значений $y(x_i)$ некоторым шумом, дисперсия которого зависит от $x$ или от номера наблюдения. 
По таким данным обучите	линейную модель и проверьте гипотезу $\mathsf{H}_0\colon \theta_2 = 0$. 
Повторите эксперимент несколько раз и посчитайте долю случаев, в которых гипотеза отвергается. Распределение шума должно быть одинаковым в каждом эксперименте.

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as sps
import scipy.special as spec
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.3)

import matplotlib
from warnings import filterwarnings
filterwarnings('ignore')
from tqdm import tqdm
from sklearn.linear_model import LinearRegression

In [3]:
thetas = np.array([1, 2, 0])
means = np.array([1, 2])
covs = np.array([1, 3])
n_samples = 10000
n_iters = 10000

In [46]:
%%time
X = sps.norm(loc=means, scale=1).rvs(size=(n_iters, n_samples, 2))
eps = sps.norm(loc=[0] * n_samples, scale=np.linspace(0.01, n_samples / 100, 
                                                              n_samples)).rvs(size=(n_iters, n_samples))
X[0], eps[0]

Wall time: 9.84 s


(array([[1.02328291, 2.09677085],
        [1.25472742, 1.07954617],
        [1.61452904, 2.58559233],
        ...,
        [0.85244254, 2.52599129],
        [1.8094729 , 1.46101151],
        [0.37403042, 3.05186689]]),
 array([-1.45399143e-02, -1.51020172e-02,  9.78534377e-03, ...,
         1.05736611e+02, -1.41933227e+02, -5.85309521e+01]))

In [48]:
res = np.array([])
for x, e in tqdm(zip(X, eps)):
    y = thetas[0] + thetas[1] * x[:, 0] + thetas[2] * x[:, 1]
    y += e
    model = LinearRegression()
    model.fit(x, y)
    y_hat = model.predict(x)
    coef_2 = model.coef_[1]
    sigma_hat = np.linalg.norm(y - y_hat) / (n_samples - 2)
    sqr = np.sqrt(np.sum(x[1] ** 2))
    t = coef_2 / sigma_hat / sqr
    pval = 2 * sps.t(df = n_samples - 2).sf(np.abs(t))
    res = np.append([res], [pval < 0.05])

10000it [00:16, 624.05it/s]


In [49]:
print(f'доля случаев, в которых гипотеза отвергается: {len(res[res == True]) / len(res)}')

доля случаев, в которых гипотеза отвергается: 0.0127


**Вывод:** в условиях гетероскедастичности, когда шум зависит линейно от номера наблюдения мы получили, что доля случаев, в которых гипотеза отвергается чуть больше 1 процента. Это означает, что реальная ошибка первого рода почти на порядок меньше допустимой, что на самом деле не очень хорошо, так как, скорее всего мы потеряли в мощности. Конечно, было бы хуже, если бы ошибка первого рода сильно превысила бы требуемый уровень значимости (что в условиях гетероскедастичности тоже могло произойти), однако в любом случае мы получили реальный уровень значимости, сильно отличающийся от требуемого, что говорит о пагубном влиянии гетероскедастичности на рассматриваемый критерий о незначимости коэффициента.